In [2]:
'''Import all the necessary packages'''
from sklearn.cross_validation import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer, OneHotEncoder, LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score

from patsy import dmatrix
from scipy import stats

import statsmodels.api as sm
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

C:\Users\win10\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\win10\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### 분석의 개요 및 목적

-주어진 Data Set 을 통해 집값에 미치는 요인들을 분석하여 러시아 집값을 예측해본다.  

### 분석의 방법 및 범위





###### EDA(탐색적 자료 분석)

- 변수 탐색 및 데이터의 이해.
- 도메인 지식 확보 및 변수별 정의.
- 산점도 확인. 
- 변수 별 상관관계 확인.

###### Preprocessing(전처리)

* 데이터 중복 제거.
* 변수 스케일링.
* 아웃라이어 탐색.

###### Model Selection(회귀 모델 설정)

* 회귀 분석 후 변수 최종 선택 및 모델 설정.
* 크로스 밸리데이션.
* 회귀 모델군 상호 평가.

### EDA
주어진 Data Set
- train.csv : 21570rows, 292columns
- test.csw : 8901rows, 291columns
- macro.csv : 2484rows, 100coumns 

#### Macro는 적용 안 하기로 결정

In [3]:
'''Import the train and test data set'''
data_train = pd.read_csv('train.csv')
data_train.shape

(21570, 292)

### Column name preprocessing 
R style 모형에서는 '-'가 설명변수 제거라는 의미를 가지고 있기 때문에 잘못 읽히는 것을 방지하기 위해 이름을 바꾸었다.
* https://www.tutorialspoint.com/python/string_replace.htm -참조